In [16]:
##%overwritefile
##%file:src/define.py
##%file:../../jupyter-MyPython-kernel/plugins/define.py
##%file:../../jupyter-MyC-kernel/plugins/define.py
##%file:../../jupyter-MyDart-kernel/plugins/define.py

##%file:../../jupyter-MyKotlin-kernel/plugins/define.py
##%file:../../jupyter-MyGroovy-kernel/plugins/define.py
##%file:../../jupyter-MyJava-kernel/plugins/define.py
##%noruncode
from typing import Dict, Tuple, Sequence,List
from plugins.ISpecialID import IStag,IDtag,IBtag,ITag
import os
import re
class Mydefine(IStag):
    kobj=None
    def getName(self) -> str:
        # self.kobj._write_to_stdout("setKernelobj setKernelobj setKernelobj\n")
        return 'Mydefine'
    def getAuthor(self) -> str:
        return 'Author'
    def getIntroduction(self) -> str:
        return 'Mydefine'
    def getPriority(self)->int:
        return 0
    def getExcludeID(self)->List[str]:
        return []
    def getIDSptag(self) -> List[str]:
        return ['define']
    def setKernelobj(self,obj):
        self.kobj=obj
        # self.kobj._write_to_stdout("setKernelobj setKernelobj setKernelobj\n")
        return
    def on_shutdown(self, restart):
        return
    def on_ISpCodescanning(self,key, value,magics,line) -> str:
        # self.kobj._write_to_stdout(line+" on_ISpCodescanning\n")
        self.kobj.addkey2dict(magics,'define')
        envdict=self.kobj._filter_env(value)
        magics[key] =dict(envdict)
        return ''
    ##生成文件时调用
    def on_before_buildfile(self,code,magics)->Tuple[bool,str]:
        return False,''
    def on_after_buildfile(self,returncode,srcfile,magics)->bool:
        return False
    def on_before_compile(self,code,magics)->Tuple[bool,str]:
        return False,''
    def on_after_compile(self,returncode,binfile,magics)->bool:
        return False
    def on_before_exec(self,code,magics)->Tuple[bool,str]:
        return False,''
    def on_after_exec(self,returncode,srcfile,magics)->bool:
        return False
    def on_after_completion(self,returncode,execfile,magics)->bool:
        return False

##%include:src/_definecodepart.py

copy to :/root/Jupyter/Myjupyter-kernel/plugins/../../jupyter-MyPython-kernel/plugins/define.py
copy to :/root/Jupyter/Myjupyter-kernel/plugins/../../jupyter-MyC-kernel/plugins/define.py
copy to :/root/Jupyter/Myjupyter-kernel/plugins/../../jupyter-MyDart-kernel/plugins/define.py
[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/plugins/src/define.py created successfully


In [4]:
##%test
##%overwritefile
##%file:src/_definecodepart.py
##test_begin
from typing import Dict, Tuple, Sequence,List
# from plugins.ISpecialID import IStag,IDtag,IBtag,ITag
import os
from jinja2 import Environment, PackageLoader, select_autoescape,Template
from typing import List, Dict, Tuple, Sequence
import re
##test_end

##test_begin
class MyMacro():
    name=''
    content=''
    lastargs={}
    def render(self,args):
        env = Environment()
        template = Template(self.content)
        ret=template.render(args)
        # print('ret'+'\n')
        return ret

class Test():
##test_end
##%include:../src/_is_specialID.py
    g_macros={}
    def macrolist(self):
        # self._log("---------MacroList--------\n")
        for key,value in self.g_macros.items():
            print( key)        
    def loadmacrofile(self,file:str):
        ##TODO 装入宏文件
        pass
    def delmacro(self,name:str):
        del g_macros[name]
    def addmacro(self,macrostr:str)->bool:
        name=''
        args=''
        content=''
        ##
        #TODO 解析 macrostr
        if self._is_specialID(macrostr):
            findObj= re.search( r':(.*)',macrostr)
            if not findObj or len(findObj.group(0))<2:
                return False
        else:
            return False
        key, value = macrostr.strip()[3:].split(":", 2)
        key = key.strip().lower()
        if key != "define":
            return False
        name,content = value.strip().split(" ", 1)
        ##
        m=MyMacro()
        m.name=name
        m.content=content
        # m.args=args
        if not self.g_macros.__contains__(name):
            d={name:None}
            self.g_macros.update(d)
        self.g_macros[name]=m
    def macrorender(self,name:str,args:Dict)->str:
        if not self.g_macros.__contains__(name):
            return ''
        m=self.g_macros[name]
        return m.render(args)
##test_begin
test=Test()
test.addmacro('##%define: m1  content {{name}} end')
test.macrolist()
ret=test.macrorender('m1',{'name':' --- '})
print(ret+'\n')
##test_end

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/plugins/src/_definecodepart.py created successfully
The process :/root/Jupyter/Myjupyter-kernel/plugins/src/_definecodepart.py


m1


 content  ---  end



The process end:6976


In [ ]:
    def _is_if(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##if') or line.strip().startswith('//if')
    def _is_else(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##else') or line.strip().startswith('//else')
    def _is_endif(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##endif') or line.strip().startswith('//endif')

    isifmacro=False
    def ifmcode(self,line):
        if not self.isifmacro:
            istb=self._is_if(line)
            if istb: 
                #登记 if
                self.isifmacro=True
                if len(line.strip())>5:
                    iste=self._is_endif(line)
                    #注销 if
                    if iste:self.isifmacro=False
                return ''
        istl=self._is_else(line):
        if istl:
            #登记 else
            # self.isifmacro=False
            return ''
        iste=self._is_endif(line)
        if iste:
            #注销 if
            self.isifmacro=False
            return ''
        line= "" if self.isifmacro else line
        return line
    
